<a href="https://colab.research.google.com/github/KendallScott/Capstone_alt/blob/main/Data_Cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:
https://geemap.org/notebooks/114_dynamic_world/


# Imports and Setup

In [17]:
!pip -q install --upgrade folium
!pip -q install geopandas
!pip -q install geojson
!pip -q install eeconvert

In [18]:
# reminder that if you are installing libraries in a Google Colab instance you will be prompted to restart your kernal

try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

In [19]:
pip install eemont

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [20]:
# Standard imports
import requests
import json

import pandas as pd
import numpy as np
from PIL import Image

# Geospatial processing packages
import geopandas as gpd
import geojson

import shapely
from shapely.geometry import box

# Mapping and plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as cl
import ee
import eeconvert as eec
import geemap
import folium
import eemont

import geopandas as gpd
from shapely.geometry import shape

### Mount Drive

In [21]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

KeyboardInterrupt: ignored

### Authenticate Google Earth Engine
Make sure you have signed up for access to Google Earth Engine at https://signup.earthengine.google.com/#!/. Once your request has been approved, you should be able to access Google Earth Engine at https://code.earthengine.google.com/.

In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:

# Read the GeoJSON file
mines = gpd.read_file('/content/drive/MyDrive/KY_Released_mines.geojson')

# Read data using GeoPandas
print("Data dimensions: {}".format(mines.shape))

In [ ]:
mines.sample(10)

In [ ]:


# Get the shape geometry
region = eec.gdfToFc(mines)

In [ ]:
mines_exploded= mines.explode()
mines_exploded

In [ ]:
# Get the shape geometry for Tennesssee mines
region = eec.gdfToFc(mines)

In [ ]:
df = pd.DataFrame()

In [ ]:
mines_exploded=mines.explode()
mines_exploded

In [ ]:
df = pd.DataFrame()
years = ['2023-07']
i=2

i_date = years[z]+'-01' # Initial date of interest (inclusive)
f_date = years[z]+'-31' # Final date of interest (exclusive)

while i<len(mines_exploded):
      fil_mines = mines_exploded.iloc[[i]]
      region = eec.gdfToFc(fil_mines)
      dc = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(region).filterDate(i_date, f_date) # Get dynamic world image collection by asset ID
      polygon = mines_exploded['geometry'].values[i]
      if polygon.geom_type == 'Polygon': # Extract the coordinates from the polygon,
        coordinates = polygon.exterior.coords[:]
        polygon = ee.Geometry.MultiPolygon(coordinates) # Create an ee.Geometry object from the coordinates
        max_error = 1 # Set the maxError parameter
        projected_polygon = polygon.transform('EPSG:4326', max_error) # Project the geometry coordinates

      elif polygon.geom_type == 'LineString':
        coordinates = polygon.coords[:]
        polygon = ee.Geometry.LineString(coordinates)

      df_contructor = []
      ac = dc.getRegion(projected_polygon, scale=200 ).getInfo() #decrease scale to get more rows of data/granularity
      df_constructor =pd.DataFrame(ac)
      df_constructor.columns = df_constructor.iloc[0]
      df_constructor = df_constructor.tail(-1)
      df_constructor['Company'] = mines_exploded['PER_NAME'].values[i]
      df = pd.concat([df, df_constructor])

print(mines_exploded['PER_NAME'].values[i])
i+=1

df =df.drop_duplicates()
df = df[df['label'].notna()]

file_name = 'ky_'+i_date+f_date
from google.colab import files
df.to_csv(file_name, encoding = 'utf-8-sig')
files.download(file_name)